In [2]:
from pandas import read_csv
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras import Model
from tensorflow.keras.layers import Input, Dense, SimpleRNN 
from tensorflow.keras.layers import Layer
from tensorflow.keras.metrics import mean_squared_error 
from tensorflow.keras.models import Sequential
import tensorflow.keras.backend as K
import numpy as np

In [3]:
def get_fib_seq(n, scale_data=True): # Get the Fibonacci sequence
    seq = np.zeros(n)
    fib_n1 = 0.0
    fib_n = 1.0
    for i in range(n):
        seq[i] = fib_n1 + fib_n
        fib_n1 = fib_n
        fib_n = seq[i]
    scaler = []
    if scale_data:
        scaler = MinMaxScaler(feature_range=(0, 1)) 
        seq = np.reshape(seq, (n, 1))
        seq = scaler.fit_transform(seq).flatten()
    return seq, scaler

fib_seq, _ = get_fib_seq(10, False)
print(fib_seq)

[ 1.  2.  3.  5.  8. 13. 21. 34. 55. 89.]


In [18]:
def get_fib_XY(total_fib_numbers, time_steps, train_percent, scale_data=True): 
    dat, scaler = get_fib_seq(total_fib_numbers, scale_data)
    Y_ind = np.arange(time_steps, len(dat), 1)
    Y = dat[Y_ind]
    rows_x = len(Y)
    X = dat[0:rows_x]
    for i in range(time_steps-1):
        temp = dat[i+1:rows_x+i+1]
    X = np.column_stack((X, temp))
    
    # random permutation with fixed seed
    rand = np.random.RandomState(seed=13)
    idx = rand.permutation(rows_x)
    split = int(train_percent*rows_x)
    train_ind = idx[0:split]
    test_ind = idx[split:]
    trainX = X[train_ind]
    trainY = Y[train_ind]
    testX = X[test_ind]
    testY = Y[test_ind]
    print(trainX)
    trainX = np.reshape(trainX, (len(trainX), time_steps, 1)) 
    testX = np.reshape(testX, (len(testX), time_steps, 1))
    return trainX, trainY, testX, testY, scaler

trainX, trainY, testX, testY, scaler = get_fib_XY(12, 2, 0.7, False) 
print('trainX = ', trainX)
print('trainY = ', trainY)


[[ 5.  8.]
 [13. 21.]
 [21. 34.]
 [ 2.  3.]
 [ 8. 13.]
 [34. 55.]
 [55. 89.]]
trainX =  [[[ 5.]
  [ 8.]]

 [[13.]
  [21.]]

 [[21.]
  [34.]]

 [[ 2.]
  [ 3.]]

 [[ 8.]
  [13.]]

 [[34.]
  [55.]]

 [[55.]
  [89.]]]
trainY =  [ 13.  34.  55.   5.  21.  89. 144.]


In [ ]:
class attention(Layer):
    def __init__(self,**kwargs):
        super().__init__(**kwargs)
    
    def build(self,input_shape):
        self.W = self.add_weight(name='attention_weight', shape=(input_shape[-1],1),
                                 initializer='random_normal', trainable=True)
        self.b = self.add_weight(name='attention_weight', shape=(input_shape[1],1),
                                initializer='zeros', trainable=True)
        super().build(input_shape)
    
    def call(self,x):
        